In [1]:
import os                                            # Importujemy moduł do interakcji z systemem operacyjnym.
import boto3                                         # Importujemy bibliotekę do obsługi protokołu S3 (DigitalOcean).
import pandas as pd                                  # Importujemy bibliotekę pandas do analizy danych.
import io                                            # Importujemy io, aby czytać dane bezpośrednio z pamięci RAM.
from dotenv import load_dotenv                       # Importujemy funkcję do wczytywania zmiennych z pliku .env.

In [2]:
load_dotenv()                                        # Ładujemy zmienne środowiskowe (klucze API).

True

In [23]:
# Lista plików do pobrania z chmury
files = [
    "data/raw/halfmarathon_wroclaw_2023__final.csv", # Ścieżka do danych z 2023 roku.
    "data/raw/halfmarathon_wroclaw_2024__final.csv"  # Ścieżka do danych z 2024 roku.
]

def get_data_from_space(file_key):                   # Definiujemy funkcję pobierającą dane.
    session = boto3.session.Session()                # Tworzymy sesję połączenia.
    client = session.client(                         # Konfigurujemy klienta pod DigitalOcean.
        's3',
        region_name='fra1',
        endpoint_url='https://fra1.digitaloceanspaces.com',
        aws_access_key_id=os.getenv('DO_SPACES_KEY'),
        aws_secret_access_key=os.getenv('DO_SPACES_SECRET')
    )
    
    response = client.get_object(Bucket=os.getenv('DO_SPACES_NAME'), Key=file_key) # Pobieramy obiekt.
    # DODAJEMY: sep=';' (separator) oraz encoding='utf-8' (polskie znaki)
    return pd.read_csv(io.BytesIO(response['Body'].read()), sep=';', encoding='utf-8') 

# Ponowne połączenie danych
df = pd.concat([get_data_from_space(f) for f in files], ignore_index=True) # Łączymy poprawione tabele.
print(f"✅ Sukces! Załadowano {len(df)} wierszy.")   

✅ Sukces! Załadowano 21957 wierszy.


In [24]:
df.sample(5)                                            # Wyświetlamy losowe 5 wierszy z załadowanego zbioru danych.

,Miejsce,Numer startowy,Imię,Nazwisko,Miasto,Kraj,Drużyna,Płeć,Płeć Miejsce,Kategoria wiekowa,...,10 km Tempo,15 km Czas,15 km Miejsce Open,15 km Tempo,20 km Czas,20 km Miejsce Open,20 km Tempo,Tempo Stabilność,Czas,Tempo
19853,NaN,7831,KRZYSZTOF,GRABOWY,NaN,NaN,NaN,M,NaN,M30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DNS,NaN
7212,7213.0,1881,BARTOSZ,KOSOWSKI,PSARY,POL,NaN,M,5353.0,M40,...,6.793333,01:42:59,7372.0,7.030000,02:21:33,7218.0,7.713333,0.061133,02:28:45,7.051434
9426,477.0,1117,BARTOSZ,FELINCZAK,RATOWICE,POL,AD GROUP,M,454.0,M40,...,4.293333,01:06:00,567.0,4.446667,01:28:37,508.0,4.523333,0.006867,01:33:28,4.430750
16245,7296.0,7730,KATARZYNA,MICHALLEK,WROCŁAW,POL,NaN,K,1592.0,K40,...,6.023333,01:31:54,7165.0,6.406667,02:06:04,7280.0,6.833333,0.060667,02:13:48,6.342735
3312,3313.0,2443,MARTA,BOBINSKA,WARSZAWA,POL,RBS,K,434.0,K30,...,5.273333,01:20:21,3290.0,5.563333,01:50:31,3285.0,6.033333,0.053800,01:56:12,5.508414


In [25]:
df.head(10)

,Miejsce,Numer startowy,Imię,Nazwisko,Miasto,Kraj,Drużyna,Płeć,Płeć Miejsce,Kategoria wiekowa,...,10 km Tempo,15 km Czas,15 km Miejsce Open,15 km Tempo,20 km Czas,20 km Miejsce Open,20 km Tempo,Tempo Stabilność,Czas,Tempo
0,1.0,1787,TOMASZ,GRYCKO,NaN,POL,UKS BLIZA WŁADYSŁAWOWO,M,1.0,M30,...,2.926667,00:44:47,1.0,3.106667,01:01:43,1.0,3.386667,0.031400,01:04:59,3.080509
1,2.0,3,ARKADIUSZ,GARDZIELEWSKI,WROCŁAW,POL,ARKADIUSZGARDZIELEWSKI.PL,M,2.0,M30,...,2.983333,00:45:26,2.0,3.143333,01:03:08,2.0,3.540000,0.038000,01:06:23,3.146875
2,3.0,3832,KRZYSZTOF,HADAS,POZNAŃ,POL,NaN,M,3.0,M20,...,3.123333,00:47:34,3.0,3.236667,01:05:09,3.0,3.516667,0.024067,01:08:24,3.242475
3,4.0,416,DAMIAN,DYDUCH,KĘPNO,POL,AZS POLITECHNIKA OPOLSKA,M,4.0,M30,...,3.196667,00:48:49,5.0,3.330000,01:06:54,4.0,3.616667,0.025467,01:10:16,3.330963
4,5.0,8476,KAMIL,MAŃKOWSKI,MIRKÓW,POL,PARKRUN WROCŁAW,M,5.0,M20,...,3.276667,00:49:31,7.0,3.386667,01:07:27,5.0,3.586667,0.023000,01:10:27,3.339654
5,6.0,2551,ADAM,PUTYRA,WROCŁAW,POL,NaN,M,6.0,M40,...,3.270000,00:49:31,6.0,3.403333,01:07:28,6.0,3.590000,0.024267,01:10:34,3.345184
6,7.0,1288,MICHAŁ,WÓJCIK,KROŚNICE,POL,WOSIEK TEAM KS AZS AWF WROCŁAW,M,7.0,M20,...,3.160000,00:48:18,4.0,3.376667,01:07:38,7.0,3.866667,0.048933,01:11:18,3.379948
7,8.0,7837,PATRYK,CHRZANOWSKI,BIELAWA,POL,PCH SPORT COMPLEX,M,8.0,M30,...,3.283333,00:49:57,8.0,3.406667,01:08:18,8.0,3.670000,0.024667,01:11:42,3.398910
8,9.0,5657,CYPRIAN,GRZELKA,POGRZEBIEŃ,POL,GKS PIAST GLIWICE,M,9.0,M20,...,3.443333,00:52:11,12.0,3.560000,01:11:03,10.0,3.773333,0.022733,01:14:16,3.520581
9,10.0,5927,ADAM,KONIECZNY,ZANIEMYSL,POL,NaN,M,10.0,M30,...,3.406667,00:51:39,10.0,3.546667,01:10:50,9.0,3.836667,0.030400,01:14:22,3.525322


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21957 entries, 0 to 21956
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Miejsce                    18450 non-null  float64
 1   Numer startowy             21957 non-null  int64  
 2   Imię                       21957 non-null  object 
 3   Nazwisko                   21957 non-null  object 
 4   Miasto                     17774 non-null  object 
 5   Kraj                       18450 non-null  object 
 6   Drużyna                    8402 non-null   object 
 7   Płeć                       21946 non-null  object 
 8   Płeć Miejsce               18450 non-null  float64
 9   Kategoria wiekowa          21926 non-null  object 
 10  Kategoria wiekowa Miejsce  18430 non-null  float64
 11  Rocznik                    21472 non-null  float64
 12  5 km Czas                  18411 non-null  object 
 13  5 km Miejsce Open          18411 non-null  flo

In [27]:
import numpy as np                                   # Importujemy numpy do obsługi wartości NaN.

In [28]:
def time_to_seconds(time_val):                       # Funkcja zamieniająca czas (tekst/float) na sekundy.
    if pd.isna(time_val) or time_val == "" or time_val == "NaN": # Sprawdzamy, czy wartość nie jest pusta.
        return None
    
    time_str = str(time_val).strip()                 # Konwertujemy na string i usuwamy białe znaki.
    try:
        parts = time_str.split(':')                  # Rozbijamy czas po dwukropku.
        if len(parts) == 3:                          # Format HH:MM:SS.
            return int(parts[0]) * 3600 + int(parts[1]) * 60 + int(parts[2])
        elif len(parts) == 2:                        # Format MM:SS.
            return int(parts[0]) * 60 + int(parts[1])
        return None                                  # Jeśli format jest inny, zwracamy None.
    except (ValueError, TypeError):                  # W przypadku błędu konwersji (np. błędny tekst).
        return None

In [29]:
# Tworzymy nowy df_ml danych dedykowaną pod trening modelu AI
df_ml = pd.DataFrame()                               # Inicjalizujemy pusty DataFrame.

# 1. Mapowanie płci: M -> 1, K -> 0 (AI preferuje liczby binarne)
df_ml['plec'] = df['Płeć'].map({'M': 1, 'K': 0})     # Zamieniamy tekstowe M/K na 1/0.

# 2. Obliczanie wieku (zakładamy rok 2024 jako bazę)
df_ml['wiek'] = 2024 - df['Rocznik']                 # Prosta operacja na roczniku.

# 3. Konwersja czasów na sekundy (nasze główne zmienne numeryczne)
df_ml['5km_s'] = df['5 km Czas'].apply(time_to_seconds) # Czas na 5km (nasz kluczowy parametr wejściowy).
df_ml['target_s'] = df['Czas'].apply(time_to_seconds)   # Czas końcowy (to, co model będzie przewidywał).

# 4. Czyszczenie końcowe - usuwamy rekordy bez kluczowych danych
df_ml = df_ml.dropna(subset=['plec', 'wiek', '5km_s', 'target_s']) # Usuwamy wiersze z brakami w tych kolumnach.

# Resetujemy indeks po usunięciu wierszy
df_ml = df_ml.reset_index(drop=True)                 # Porządkujemy numerację wierszy.

In [30]:
print(f"✅ Po wyczyszczeniu zostało {len(df_ml)} rekordów.") # Informujemy o wielkości zbioru do nauki.
print(df_ml.head(10))                                  # Wyświetlamy podgląd gotowych danych.

✅ Po wyczyszczeniu zostało 17927 rekordów.
   plec  wiek   5km_s  target_s
0   1.0  32.0   877.0    3899.0
1   1.0  38.0   888.0    3983.0
2   1.0  28.0   946.0    4104.0
3   1.0  36.0   971.0    4216.0
4   1.0  29.0   972.0    4227.0
5   1.0  41.0   969.0    4234.0
6   1.0  25.0   937.0    4278.0
7   1.0  35.0   990.0    4302.0
8   1.0  23.0  1030.0    4456.0
9   1.0  32.0  1013.0    4462.0


In [ ]:
from pycaret.regression import setup, compare_models, finalize_model, save_model    # Importujemy funkcje regresji z PyCaret.
import os                                                                           # Importujemy os do zarządzania ścieżkami plików.

In [32]:
# 1. Konfiguracja środowiska PyCaret
# target='target_s' - wskazujemy co chcemy przewidzieć. 
# session_id=123 - ustawiamy ziarno losowości dla powtarzalności wyników.
s = setup(data=df_ml, target='target_s', session_id=123, verbose=False) # Inicjalizujemy eksperyment.

In [33]:
# 2. Porównanie modeli i wybór najlepszego
# Jako metrykę wybieramy MAE (Mean Absolute Error), bo łatwo ją zinterpretować (o ile sekund myli się model).
best_model = compare_models(sort='MAE')              # Trenujemy zestaw modeli i sortujemy wg błędu MAE.

# 3. Finalizacja modelu
# Trenujemy wybrany najlepszy model na całym dostępnym zbiorze danych (df_ml).
final_marathon_model = finalize_model(best_model)    # Tworzymy ostateczną wersję modelu.

# 4. Zapis lokalny
model_name = 'marathon_model_v1'                     # Definiujemy nazwę pliku modelu.
save_model(final_marathon_model, model_name)         # PyCaret zapisze plik jako marathon_model_v1.pkl.

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
huber,Huber Regressor,297.8067,181340.7615,425.3321,0.8761,0.0530,0.0385,0.0240
gbr,Gradient Boosting Regressor,298.8487,170204.1058,412.1732,0.8836,0.0522,0.0391,0.1230
omp,Orthogonal Matching Pursuit,300.8259,177175.7453,420.3514,0.8790,0.0526,0.0393,0.0150
br,Bayesian Ridge,300.8944,177222.4281,420.4074,0.8790,0.0526,0.0393,0.0100
en,Elastic Net,300.8958,177223.7000,420.4089,0.8790,0.0526,0.0393,0.0100
llar,Lasso Least Angle Regression,300.8987,177226.6703,420.4126,0.8790,0.0526,0.0393,0.0090
lasso,Lasso Regression,300.8988,177226.6703,420.4126,0.8790,0.0526,0.0393,0.0120
lr,Linear Regression,300.9304,177228.7188,420.4158,0.8790,0.0526,0.0393,0.7270
ridge,Ridge Regression,300.9304,177228.7219,420.4158,0.8790,0.0526,0.0393,0.0100
lar,Least Angle Regression,300.9305,177228.7234,420.4158,0.8790,0.0526,0.0393,0.0090


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['plec', 'wiek', '5km_s'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=[],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('actual_estimator', HuberRegressor())]),
 'marathon_model_v1.pkl')

### Komentarz do bloku: > * Metryki: Używamy MAE (Średni Błąd Bezwzględny). Jeśli MAE wyniesie 300, oznacza to, że model myli się średnio o 5 minut (300 sekund). To bardzo czytelne dla biegacza.

In [34]:
# 5. Upload modelu do DigitalOcean Spaces
def upload_model_to_spaces(file_path, bucket):       # Funkcja wysyłająca plik modelu do chmury.
    client = boto3.session.Session().client(         # Inicjalizujemy klienta S3.
        's3',
        region_name='fra1',
        endpoint_url='https://fra1.digitaloceanspaces.com',
        aws_access_key_id=os.getenv('DO_SPACES_KEY'),
        aws_secret_access_key=os.getenv('DO_SPACES_SECRET')
    )
    
    remote_path = f"models/{file_path}.pkl"          # Określamy ścieżkę w chmurze w folderze /models.
    client.upload_file(f"{file_path}.pkl", bucket, remote_path) # Przesyłamy plik.
    print(f"🚀 Model {file_path} wysłany do Spaces: {remote_path}") # Informujemy o sukcesie.

upload_model_to_spaces(model_name, os.getenv('DO_SPACES_NAME')) # Wywołujemy upload.

🚀 Model marathon_model_v1 wysłany do Spaces: models/marathon_model_v1.pkl
